# HashtagTrainamentos  - Datas Personalizadas no Power BI - Mês, Bimestre, Semestre

https://www.youtube.com/watch?v=3YZ_awh57dY

In [1]:
#import libs
import pandas as pd

In [7]:
#!pip install xlrd
#!pip install openpyxl 

Using legacy 'setup.py install' for et-xmlfile, since package 'wheel' is not installed.
    Running setup.py install for et-xmlfile: started
    Running setup.py install for et-xmlfile: finished with status 'done'


In [58]:
df = pd.read_excel('Base Vendas.xlsx')

df.head()

,SKU,Qtd Vendida,Loja,Data da Venda,Preco Unitario,Faturamento,Produto
0,HL1018,5,Porto Alegre,2016-01-01,1200,6000,Smart Watch
1,HL1001,2,Fortaleza,2016-01-01,2600,5200,Televisão
2,HL1008,1,Rio de Janeiro,2016-01-01,2000,2000,Televisão
3,HL1021,1,Recife,2016-01-01,3000,3000,Celular
4,HL1008,5,Salvador,2016-01-01,2000,10000,Televisão


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23794 entries, 0 to 23793
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   SKU             23794 non-null  object        
 1   Qtd Vendida     23794 non-null  int64         
 2   Loja            23794 non-null  object        
 3   Data da Venda   23794 non-null  datetime64[ns]
 4   Preco Unitario  23794 non-null  int64         
 5   Faturamento     23794 non-null  int64         
 6   Produto         23794 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(3)
memory usage: 1.3+ MB


In [60]:
def create_calendar(start='2000-01-01', end='2022-12-31'):
    df = pd.DataFrame({"Date": pd.date_range(start, end)})
    df["Day"] = df.Date.dt.day
    df["Week"] = df.Date.dt.isocalendar().week
    df["Week_day"] = df.Date.dt.weekday
    df["Month"] = df.Date.dt.month
    df["Month_name"] = df.Date.dt.month_name()
    df["Month_abbr"] = df.Date.dt.month_name().str[:3]
    df["Quarter"] = df.Date.dt.quarter
    df["Year"] = df.Date.dt.year
    df["Year_half"] = (df.Quarter + 1) // 2
    return df

In [61]:
dCalendario = create_calendar(df['Data da Venda'].min(), df['Data da Venda'].max())
dCalendario

,Date,Day,Week,Week_day,Month,Month_name,Month_abbr,Quarter,Year,Year_half
0,2016-01-01,1,53,4,1,January,Jan,1,2016,1
1,2016-01-02,2,53,5,1,January,Jan,1,2016,1
2,2016-01-03,3,53,6,1,January,Jan,1,2016,1
3,2016-01-04,4,1,0,1,January,Jan,1,2016,1
4,2016-01-05,5,1,1,1,January,Jan,1,2016,1
...,...,...,...,...,...,...,...,...,...,...
1091,2018-12-27,27,52,3,12,December,Dec,4,2018,2
1092,2018-12-28,28,52,4,12,December,Dec,4,2018,2
1093,2018-12-29,29,52,5,12,December,Dec,4,2018,2
1094,2018-12-30,30,52,6,12,December,Dec,4,2018,2


In [71]:
print(f"Valor do faturamento total: R$ {df['Faturamento'].sum():,.2f}")

Valor do faturamento total: R$ 156,765,700.00


In [99]:
# calculo do faturamento anual
# relacionando a tabela calendario e faturamento
df_temp = pd.merge(dCalendario[['Date', 'Year', 'Month', 'Month_abbr']], df[['Data da Venda', 'Faturamento']], how='left', left_on='Date', right_on='Data da Venda')
df_temp.groupby('Year')['Faturamento'].sum().map('R$ {:,.2f}'.format)

Year
2016    R$ 39,630,500.00
2017    R$ 50,817,800.00
2018    R$ 66,317,400.00
Name: Faturamento, dtype: object

In [108]:
# calculo do faturamento mensal
# relacionando a tabela calendario e faturamento
format_mapping = {'Faturamento': 'R$ {:,.2f}'}
df_temp.groupby(['Year', 'Month', 'Month_abbr'], as_index=False)['Faturamento'].sum().sort_values(['Year', 'Month', 'Month_abbr']).drop(columns='Month').style.format(format_mapping)

,Year,Month_abbr,Faturamento
0,2016,Jan,"R$ 4,234,650.00"
1,2016,Feb,"R$ 2,839,300.00"
2,2016,Mar,"R$ 2,421,250.00"
3,2016,Apr,"R$ 3,120,700.00"
4,2016,May,"R$ 2,517,200.00"
5,2016,Jun,"R$ 3,130,600.00"
6,2016,Jul,"R$ 4,515,550.00"
7,2016,Aug,"R$ 2,839,800.00"
8,2016,Sep,"R$ 3,071,150.00"
9,2016,Oct,"R$ 2,650,500.00"


In [110]:
# calculo do faturamento mensal
# relacionando a tabela calendario e faturamento
format_mapping = {'Faturamento': 'R$ {:,.2f}'}
df_temp = pd.merge(dCalendario[['Date', 'Year', 'Month', 'Month_name']], df[['Data da Venda', 'Faturamento']], how='left', left_on='Date', right_on='Data da Venda')
df_temp.groupby(['Year', 'Month', 'Month_name'], as_index=False)['Faturamento'].sum().sort_values(['Year', 'Month', 'Month_name']).drop(columns='Month').style.format(format_mapping)

,Year,Month_name,Faturamento
0,2016,January,"R$ 4,234,650.00"
1,2016,February,"R$ 2,839,300.00"
2,2016,March,"R$ 2,421,250.00"
3,2016,April,"R$ 3,120,700.00"
4,2016,May,"R$ 2,517,200.00"
5,2016,June,"R$ 3,130,600.00"
6,2016,July,"R$ 4,515,550.00"
7,2016,August,"R$ 2,839,800.00"
8,2016,September,"R$ 3,071,150.00"
9,2016,October,"R$ 2,650,500.00"


In [112]:
# calculo do faturamento por trimestre "quarter"
# relacionando a tabela calendario e faturamento
format_mapping = {'Quarter': 'Q{}', 'Faturamento': 'R$ {:,.2f}'}
df_temp = pd.merge(dCalendario[['Date', 'Year', 'Quarter']], df[['Data da Venda', 'Faturamento']], how='left', left_on='Date', right_on='Data da Venda')
df_temp.groupby(['Year', 'Quarter'], as_index=False)['Faturamento'].sum().sort_values(['Year', 'Quarter']).style.format(format_mapping)

,Year,Quarter,Faturamento
0,2016,Q1,"R$ 9,495,200.00"
1,2016,Q2,"R$ 8,768,500.00"
2,2016,Q3,"R$ 10,426,500.00"
3,2016,Q4,"R$ 10,940,300.00"
4,2017,Q1,"R$ 11,329,350.00"
5,2017,Q2,"R$ 10,028,800.00"
6,2017,Q3,"R$ 14,144,400.00"
7,2017,Q4,"R$ 15,315,250.00"
8,2018,Q1,"R$ 15,409,350.00"
9,2018,Q2,"R$ 14,531,250.00"


In [114]:
# calculo do faturamento por semestre
# relacionando a tabela calendario e faturamento
format_mapping = {'Year_half': 'S{}', 'Faturamento': 'R$ {:,.2f}'}
df_temp = pd.merge(dCalendario[['Date', 'Year', 'Year_half']], df[['Data da Venda', 'Faturamento']], how='left', left_on='Date', right_on='Data da Venda')
df_temp.groupby(['Year', 'Year_half'], as_index=False)['Faturamento'].sum().sort_values(['Year', 'Year_half']).style.format(format_mapping)

,Year,Year_half,Faturamento
0,2016,S1,"R$ 18,263,700.00"
1,2016,S2,"R$ 21,366,800.00"
2,2017,S1,"R$ 21,358,150.00"
3,2017,S2,"R$ 29,459,650.00"
4,2018,S1,"R$ 29,940,600.00"
5,2018,S2,"R$ 36,376,800.00"
